In [2]:
from collections import defaultdict, Counter
import time
import random
import torch
import json
 
x_text = []
y = []
with open('reviews_Musical_Instruments_5.json\Musical_Instruments_5.json') as f:
    for line in f:
        data = json.loads(line)
        x_text.append(data['reviewText'].lower().strip())
        y.append(int(data['overall']))

In [3]:
x_text = x_text[:1000]
y = y[:1000]
train_end_idx=int(0.9 * len(y))

In [5]:
w2i = defaultdict(lambda: len(w2i))
UNK = w2i["<unk>"] 


t = []
for x in x_text:
    t += x.split(' ')

for w, n in Counter(t).items():
    if n > 5:
        w2i[w]
n_words = len(w2i)

In [14]:
w2i = defaultdict(lambda: UNK, w2i) 
class2i = defaultdict(lambda: len(class2i))
    
def read_dataset(start_idx,end_idx):
    for i, text in enumerate(x_text[start_idx:end_idx]):
        yield ([w2i[x] for x in text.split(" ")], class2i[y[i]])
        
train = list(read_dataset(0, train_end_idx))
dev = list(read_dataset(train_end_idx, len(y)))
n_class = len(class2i)

In [7]:
print(n_words, n_class)

1389 5


In [9]:
EMBEDDING_SIZE = 20
C = torch.rand((n_words, EMBEDDING_SIZE), requires_grad=True)
W = torch.rand((n_class, EMBEDDING_SIZE+1), requires_grad=True)



In [37]:
class SimpleModel(torch.nn.Module):
    def __init__(self, n_words, emb_size, n_class):
        super(SimpleModel, self).__init__()
        self.embedding = torch.nn.Embedding(n_words, emb_size)
        self.linear = torch.nn.Linear(in_features=emb_size, out_features=n_class, bias=True)
        torch.nn.init.uniform_(self.embedding.weight, -0.25, 0.25)
        torch.nn.init.xavier_uniform_(self.linear.weight)

    def forward(self, words):
        emb = self.embedding(words)                 
        h = emb.mean(dim=0)                         
        h = torch.reshape(h, (1,-1))
        out = self.linear(h)              
        return out


In [41]:
model = SimpleModel(n_words, EMBEDDING_SIZE, n_class)
print([i for i in model.parameters()])

[Parameter containing:
tensor([[-0.0006,  0.0624,  0.0098,  ...,  0.1398, -0.0790,  0.2460],
        [-0.1673,  0.2464, -0.0037,  ..., -0.2457, -0.0821,  0.2042],
        [-0.2050,  0.1472,  0.0635,  ...,  0.1149,  0.1142,  0.0938],
        ...,
        [ 0.1722, -0.1179, -0.1573,  ...,  0.2258,  0.0708, -0.1116],
        [-0.0057,  0.1040,  0.1037,  ...,  0.1357, -0.1066, -0.0345],
        [-0.0880, -0.0358,  0.1797,  ..., -0.1048, -0.0637,  0.1526]],
       requires_grad=True), Parameter containing:
tensor([[ 0.3197, -0.1474, -0.1573,  0.1251, -0.0480,  0.4300,  0.1106,  0.1935,
         -0.3719,  0.0290,  0.0591, -0.4199, -0.1380, -0.1446,  0.1038,  0.4848,
         -0.3397, -0.0840, -0.1847,  0.0368],
        [-0.4550,  0.3454,  0.0437, -0.2554, -0.4364,  0.2466,  0.0495, -0.0443,
         -0.3265,  0.4659, -0.4587,  0.4723,  0.3461,  0.2374,  0.1582,  0.4053,
          0.4277, -0.0298,  0.4010,  0.2092],
        [ 0.2309,  0.3550,  0.1285,  0.0266,  0.2219,  0.1832,  0.1480, -0.38

In [45]:
epochs = 5
eta = 0.5  # prędkość uczenia

for i in range(epochs):
    random.shuffle(train)
    train_loss = 0.0
    for words, tag in train:
        pred = model(torch.tensor(words))
        loss = F.cross_entropy(pred.reshape(1,-1), torch.tensor(tag).reshape(1))
        loss.backward()
        with torch.no_grad():
            for p in model.parameters():
                p -= eta * p.grad
            model.zero_grad()
        train_loss += loss
    print("iter %r: avg. train loss=%.4f" % (i, train_loss / len(train)))

iter 0: avg. train loss=0.8431
iter 1: avg. train loss=0.8105
iter 2: avg. train loss=0.7889
iter 3: avg. train loss=0.7576
iter 4: avg. train loss=0.7366


In [46]:
from torch import optim
epochs = 5
eta = 0.5  

optimizer = torch.optim.SGD(model.parameters(), lr=eta)

for i in range(epochs):
    random.shuffle(train)
    train_loss = 0.0
    for words, tag in train:
        pred = model(torch.tensor(words))
        loss = F.cross_entropy(pred.view(1,-1), torch.tensor([tag]))
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        train_loss += loss
    print("iter %r: avg. train loss=%.4f" % (i, train_loss / len(train)))

iter 0: avg. train loss=0.6740
iter 1: avg. train loss=0.6522
iter 2: avg. train loss=0.6149
iter 3: avg. train loss=0.6004
iter 4: avg. train loss=0.5691


**CNN**

In [20]:
class CNN(torch.nn.Module):
    def __init__(self, n_words, emb_size, num_filters, window_size, ntags):
        super().__init__()
        self.emb_size, self.num_filters = emb_size, num_filters
        self.emb = torch.nn.Embedding(n_words, emb_size)
        self.conv = torch.nn.Conv1d(emb_size, num_filters, kernel_size=window_size)
        self.lin = torch.nn.Linear(in_features=num_filters, out_features=ntags, bias=True)
        for layer in [self.emb, self.conv, self.lin]:
            torch.nn.init.xavier_uniform_(layer.weight)

    def forward(self, words):
        x = words
        x = self.emb(x).reshape(1, self.emb_size, -1)  
        x = self.conv(x).reshape(self.num_filters, -1) 
        x = F.relu(x).max(1).values                 
        x = self.lin(x.reshape(-1))                    
        return x
       
def train_loop(model, epochs=5, lr=0.05):
    optim = torch.optim.SGD(model.parameters(), lr=lr)
    for i in range(epochs):
        random.shuffle(train)
        train_loss = 0.0
        for words, tag in train:
            pred = model(torch.tensor(words))
            loss = F.cross_entropy(pred.view(1,-1), torch.tensor([tag]))
            loss.backward()
            train_loss += loss
            optim.step()
            optim.zero_grad()
        print("iter %r: avg. train loss=%.4f" % (i, train_loss / len(train)))
    return model
        
model = CNN(n_words=n_words, emb_size=EMBEDDING_SIZE, ntags=n_class, num_filters=64, window_size=2)
train_loop(model, epochs=20, lr=0.03)

iter 0: avg. train loss=0.9462
iter 1: avg. train loss=0.9245
iter 2: avg. train loss=0.9037
iter 3: avg. train loss=0.9051
iter 4: avg. train loss=0.8956
iter 5: avg. train loss=0.8679
iter 6: avg. train loss=0.8446
iter 7: avg. train loss=0.7814
iter 8: avg. train loss=0.7039
iter 9: avg. train loss=0.5822
iter 10: avg. train loss=0.4217
iter 11: avg. train loss=0.2830
iter 12: avg. train loss=0.1830
iter 13: avg. train loss=0.1127
iter 14: avg. train loss=0.0738
iter 15: avg. train loss=0.0477
iter 16: avg. train loss=0.0319
iter 17: avg. train loss=0.0221
iter 18: avg. train loss=0.0160
iter 19: avg. train loss=0.0118


CNN(
  (emb): Embedding(1389, 20)
  (conv): Conv1d(20, 64, kernel_size=(2,), stride=(1,))
  (lin): Linear(in_features=64, out_features=5, bias=True)
)